In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from pandas.plotting import scatter_matrix
import seaborn as sns
from IPython.display import set_matplotlib_formats, HTML
from matplotlib.dates import DateFormatter
import matplotlib_inline 
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from matplotlib import colors as mcolors
from pandas.plotting import register_matplotlib_converters
import plotly.express as px
%matplotlib inline
%config InlineBackend.figure_format = 'png'
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore') 

In [2]:
# Formating Plots
# default styles
def set_sns_format(width=14, height=8):
    sns.set_theme(palette='pastel', context='notebook',rc={'savefig.dpi':300})
    matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
    matplotlib.rcParams['figure.figsize'] = (width, height)
    return None
set_sns_format(width=14, height=8)

In [3]:
def add_value_labels(ax, typ, spacing=5):
    #This function add the labels in the bar and line plots
    #input the ax to add the labels, the type of plot
    
    space = spacing
    va = 'bottom'
    

    if typ == 'bar':
        for i in ax.patches:
            y_value = i.get_height()
            x_value = i.get_x() + i.get_width() / 2

            label = "{:.2f}".format(y_value)
            ax.annotate(label,(x_value, y_value), xytext=(0, space), 
                    textcoords="offset points", ha='center', va=va, fontsize=10)     

    if typ == 'line':
        for line in ax.lines:
            for x_value, y_value in zip(line.get_xdata(), line.get_ydata()):
                label = "{:.2f}".format(y_value)
                ax.annotate(label,(x_value, y_value), xytext=(0, space), 
                    textcoords="offset points", ha='center', va=va, fontsize=10)

In [4]:
from mysql.connector import connect
from getpass import getpass

In [5]:
db = connect(
host = 'localhost',
user = 'root',
passwd = getpass('Enter password:'),
database = 'sakila')

## Query the SQL data to Pandas

In [6]:
cursor = db.cursor()
query1 = 'SELECT f.film_id,f.title,f.release_year,f.language_id,f.rental_rate,f.length, f.rating, f.special_features, DATE_FORMAT(MAX(r.rental_date), \"%y\") AS last_rental_year, DATE_FORMAT(MAX(r.rental_date), \"%m\") AS last_rental_month,COUNT(r.rental_id) AS rentals FROM film f LEFT JOIN inventory i ON f.film_id = i.film_id LEFT JOIN rental r ON i.inventory_id = r.inventory_id GROUP BY f.film_id;'
cursor.execute(query1)

In [7]:
database = pd.DataFrame(cursor.fetchall())
database

,0,1,2,3,4,5,6,7,8,9,10
0,1,ACADEMY DINOSAUR,2006,1,0.99,86,PG,"{Behind the Scenes, Deleted Scenes}",05,08,23
1,2,ACE GOLDFINGER,2006,1,4.99,48,G,"{Deleted Scenes, Trailers}",06,02,7
2,3,ADAPTATION HOLES,2006,1,2.99,50,NC-17,"{Deleted Scenes, Trailers}",05,08,12
3,4,AFFAIR PREJUDICE,2006,1,2.99,117,G,"{Commentaries, Behind the Scenes}",06,02,23
4,5,AFRICAN EGG,2006,1,2.99,130,G,{Deleted Scenes},06,02,12
...,...,...,...,...,...,...,...,...,...,...,...
995,996,YOUNG LANGUAGE,2006,1,0.99,183,G,"{Behind the Scenes, Trailers}",05,08,7
996,997,YOUTH KICK,2006,1,0.99,179,NC-17,"{Behind the Scenes, Trailers}",05,08,6
997,998,ZHIVAGO CORE,2006,1,0.99,105,NC-17,{Deleted Scenes},06,02,9
998,999,ZOOLANDER FICTION,2006,1,2.99,101,R,"{Deleted Scenes, Trailers}",05,08,17


### Formating the Data and Creating Target Variable.

In [8]:
database.rename(columns={ 8 :"year", 9:"month"}, inplace = True, errors = "raise")
database.columns.to_list()

[0, 1, 2, 3, 4, 5, 6, 7, 'year', 'month', 10]

In [9]:
database["rented_last_month"] = np.where((database["year"] == "06") & (database["month"] == "02"), 1, 0)
database

,0,1,2,3,4,5,6,7,year,month,10,rented_last_month
0,1,ACADEMY DINOSAUR,2006,1,0.99,86,PG,"{Behind the Scenes, Deleted Scenes}",05,08,23,0
1,2,ACE GOLDFINGER,2006,1,4.99,48,G,"{Deleted Scenes, Trailers}",06,02,7,1
2,3,ADAPTATION HOLES,2006,1,2.99,50,NC-17,"{Deleted Scenes, Trailers}",05,08,12,0
3,4,AFFAIR PREJUDICE,2006,1,2.99,117,G,"{Commentaries, Behind the Scenes}",06,02,23,1
4,5,AFRICAN EGG,2006,1,2.99,130,G,{Deleted Scenes},06,02,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,YOUNG LANGUAGE,2006,1,0.99,183,G,"{Behind the Scenes, Trailers}",05,08,7,0
996,997,YOUTH KICK,2006,1,0.99,179,NC-17,"{Behind the Scenes, Trailers}",05,08,6,0
997,998,ZHIVAGO CORE,2006,1,0.99,105,NC-17,{Deleted Scenes},06,02,9,1
998,999,ZOOLANDER FICTION,2006,1,2.99,101,R,"{Deleted Scenes, Trailers}",05,08,17,0


In [10]:
database.rename({1 : "title", 2 : "release_year", 3 : "language",  4 : "rental_rate", 5 : "length", 6 : "rating", 7 : "special_features", 10 : "rentals"},inplace = True, axis = 1)
database.drop(0, axis=1, inplace = True)

In [11]:
database

,title,release_year,language,rental_rate,length,rating,special_features,year,month,rentals,rented_last_month
0,ACADEMY DINOSAUR,2006,1,0.99,86,PG,"{Behind the Scenes, Deleted Scenes}",05,08,23,0
1,ACE GOLDFINGER,2006,1,4.99,48,G,"{Deleted Scenes, Trailers}",06,02,7,1
2,ADAPTATION HOLES,2006,1,2.99,50,NC-17,"{Deleted Scenes, Trailers}",05,08,12,0
3,AFFAIR PREJUDICE,2006,1,2.99,117,G,"{Commentaries, Behind the Scenes}",06,02,23,1
4,AFRICAN EGG,2006,1,2.99,130,G,{Deleted Scenes},06,02,12,1
...,...,...,...,...,...,...,...,...,...,...,...
995,YOUNG LANGUAGE,2006,1,0.99,183,G,"{Behind the Scenes, Trailers}",05,08,7,0
996,YOUTH KICK,2006,1,0.99,179,NC-17,"{Behind the Scenes, Trailers}",05,08,6,0
997,ZHIVAGO CORE,2006,1,0.99,105,NC-17,{Deleted Scenes},06,02,9,1
998,ZOOLANDER FICTION,2006,1,2.99,101,R,"{Deleted Scenes, Trailers}",05,08,17,0


In [12]:
database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   title              1000 non-null   object
 1   release_year       1000 non-null   int64 
 2   language           1000 non-null   int64 
 3   rental_rate        1000 non-null   object
 4   length             1000 non-null   int64 
 5   rating             1000 non-null   object
 6   special_features   1000 non-null   object
 7   year               958 non-null    object
 8   month              958 non-null    object
 9   rentals            1000 non-null   int64 
 10  rented_last_month  1000 non-null   int32 
dtypes: int32(1), int64(4), object(6)
memory usage: 82.2+ KB


In [13]:
database.release_year.value_counts()

2006    1000
Name: release_year, dtype: int64

In [14]:
database.language.value_counts()

1    1000
Name: language, dtype: int64

In [15]:
database.drop(["title", "release_year", "language"], axis=1, inplace = True)
database

,rental_rate,length,rating,special_features,year,month,rentals,rented_last_month
0,0.99,86,PG,"{Behind the Scenes, Deleted Scenes}",05,08,23,0
1,4.99,48,G,"{Deleted Scenes, Trailers}",06,02,7,1
2,2.99,50,NC-17,"{Deleted Scenes, Trailers}",05,08,12,0
3,2.99,117,G,"{Commentaries, Behind the Scenes}",06,02,23,1
4,2.99,130,G,{Deleted Scenes},06,02,12,1
...,...,...,...,...,...,...,...,...
995,0.99,183,G,"{Behind the Scenes, Trailers}",05,08,7,0
996,0.99,179,NC-17,"{Behind the Scenes, Trailers}",05,08,6,0
997,0.99,105,NC-17,{Deleted Scenes},06,02,9,1
998,2.99,101,R,"{Deleted Scenes, Trailers}",05,08,17,0


In [16]:
database = database.fillna(value = 0, axis = 0)

In [17]:
database.rental_rate = pd.to_numeric(database.rental_rate)
database.year = database.year.astype(int)
database.month = database.month.astype(int)
database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rental_rate        1000 non-null   float64
 1   length             1000 non-null   int64  
 2   rating             1000 non-null   object 
 3   special_features   1000 non-null   object 
 4   year               1000 non-null   int32  
 5   month              1000 non-null   int32  
 6   rentals            1000 non-null   int64  
 7   rented_last_month  1000 non-null   int32  
dtypes: float64(1), int32(3), int64(2), object(2)
memory usage: 50.9+ KB


In [18]:
database

,rental_rate,length,rating,special_features,year,month,rentals,rented_last_month
0,0.99,86,PG,"{Behind the Scenes, Deleted Scenes}",5,8,23,0
1,4.99,48,G,"{Deleted Scenes, Trailers}",6,2,7,1
2,2.99,50,NC-17,"{Deleted Scenes, Trailers}",5,8,12,0
3,2.99,117,G,"{Commentaries, Behind the Scenes}",6,2,23,1
4,2.99,130,G,{Deleted Scenes},6,2,12,1
...,...,...,...,...,...,...,...,...
995,0.99,183,G,"{Behind the Scenes, Trailers}",5,8,7,0
996,0.99,179,NC-17,"{Behind the Scenes, Trailers}",5,8,6,0
997,0.99,105,NC-17,{Deleted Scenes},6,2,9,1
998,2.99,101,R,"{Deleted Scenes, Trailers}",5,8,17,0


In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
database.rating = le.fit_transform(database.rating)
database.special_features = database.special_features.astype("string")
database.special_features = le.fit_transform(database.special_features)

In [20]:
# Separation X and y 
x = database[database.columns.drop("rented_last_month")] # features
x = x.drop(columns = ["year", "month"])              # these were used to create the target feature(drop because of collinearity)
y = database["rented_last_month"]                    # target feature

In [21]:
# Train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, random_state = 42)

In [22]:
# linear regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
reg =  LinearRegression()   


X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test)       
reg.fit(X_train, y_train)

LinearRegression()

In [23]:
from sklearn.metrics import r2_score
predictions_train = reg.predict(X_train) # create predictions for our train data
predictions_test = reg.predict(X_test)   # create predictions for our test data

r2_train = r2_score(y_train, predictions_train) # calculate r2 score for train data
r2_test = r2_score(y_test, predictions_test)    # calculate r2 score for test data

print('R2 value for train: {}'.format(r2_train))
print('R2 value for test: {}'.format(r2_test)) 

R2 value for train: 0.04353510383117187
R2 value for test: -0.026242371956933574


In [24]:
feature_importance = pd.DataFrame ({
    "feature": x.columns,
    'values': reg.coef_
})
feature_importance.sort_values(['values'], ascending = False)

,feature,values
4,rentals,0.074174
3,special_features,0.009341
1,length,-0.008861
0,rental_rate,-0.011954
2,rating,-0.022607


##  This model is not useful. we need more features